https://programmer.help/blogs/fifty-sql-exercise-questions-and-answers-and-detailed-analysis.html

In [2]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%load_ext sql

# Verbinde Dich zu einer in - Memory Datenbank
%sql sqlite:///uni.db
%sql PRAGMA foreign_keys = ON

 * sqlite:///uni.db
Done.


[]

In [10]:
# !echo .schema | sqlite3 uni.db

# CREATE TABLE Studenten
#        (MatrNr         INTEGER PRIMARY KEY,
#         Name           VARCHAR(30) NOT NULL,
#         Semester       INTEGER);
# CREATE TABLE Professoren
#        (PersNr         INTEGER PRIMARY KEY,
#         Name           VARCHAR(30) NOT NULL,
#         Rang           CHAR(2) CHECK (Rang in ('C2', 'C3', 'C4')),
#         Raum           INTEGER UNIQUE);
# CREATE TABLE Assistenten
#        (PersNr         INTEGER PRIMARY KEY,
#         Name           VARCHAR(30) NOT NULL,
#         Fachgebiet     VARCHAR(30),
#         Boss           INTEGER,
#         FOREIGN KEY    (Boss) REFERENCES Professoren);
# CREATE TABLE Vorlesungen
#        (VorlNr         INTEGER PRIMARY KEY,
#         Titel          VARCHAR(30),
#         SWS            INTEGER,
#         gelesenVon     INTEGER REFERENCES Professoren);
# CREATE TABLE hoeren
#        (MatrNr         INTEGER REFERENCES Studenten ON DELETE CASCADE,
#         VorlNr         INTEGER REFERENCES Vorlesungen ON DELETE CASCADE,
#         PRIMARY KEY    (MatrNr, VorlNr));
# CREATE TABLE voraussetzen
#        (Vorgaenger     INTEGER REFERENCES Vorlesungen ON DELETE CASCADE,
#         Nachfolger     INTEGER REFERENCES Vorlesungen ON DELETE NO ACTION,
#         PRIMARY KEY    (Vorgaenger, Nachfolger));
# CREATE TABLE pruefen
#        (MatrNr         INTEGER REFERENCES Studenten ON DELETE CASCADE,
#         VorlNr         INTEGER REFERENCES Vorlesungen,
#         PersNr         INTEGER REFERENCES Professoren,
#         Note           NUMERIC(2,1) CHECK (Note between 0.7 and 5.0),
#         PRIMARY KEY    (MatrNr, VorlNr));

In [3]:
%%sql

SELECT * FROM Studenten

 * sqlite:///uni.db
Done.


MatrNr,Name,Semester
24002,Xenokrates,18
25403,Jonas,12
26120,Fichte,10
26830,Aristoxenos,8
27550,Schopenhauer,6
28106,Carnap,3
29120,Theophrastos,2
29555,Feuerbach,2


In [4]:
%%sql

SELECT * FROM Professoren

 * sqlite:///uni.db
Done.


PersNr,Name,Rang,Raum
2125,Sokrates,C4,226
2126,Russel,C4,232
2127,Kopernikus,C3,310
2133,Popper,C3,52
2134,Augustinus,C3,309
2136,Curie,C4,36
2137,Kant,C4,7


In [5]:
%%sql

SELECT * FROM Assistenten

 * sqlite:///uni.db
Done.


PersNr,Name,Fachgebiet,Boss
3002,Platon,Ideenlehre,2125
3003,Aristoteles,Syllogistik,2125
3004,Wittgenstein,Sprachtheorie,2126
3005,Rhetikus,Planetenbewegung,2127
3006,Newton,Keplersche Gesetze,2127
3007,Spinoza,Gott und Natur,2134


In [6]:
%%sql

SELECT * FROM Vorlesungen

 * sqlite:///uni.db
Done.


VorlNr,Titel,SWS,gelesenVon
4052,Logik,4,2125
4630,Die 3 Kritiken,4,2137
5001,Grundzuege,4,2137
5022,Glaube und Wissen,2,2134
5041,Ethik,4,2125
5043,Erkenntnistheorie,3,2126
5049,Maeeutik,2,2125
5052,Wissenschaftstheorie,3,2126
5216,Bioethik,2,2126
5259,Der Wiener Kreis,2,2133


In [7]:
%%sql

SELECT * FROM hoeren

 * sqlite:///uni.db
Done.


MatrNr,VorlNr
26120,5001
27550,5001
27550,4052
28106,5041
28106,5052
28106,5216
28106,5259
29120,5001
29120,5041
29120,5049


In [8]:
%%sql

SELECT * FROM voraussetzen

 * sqlite:///uni.db
Done.


Vorgaenger,Nachfolger
5001,5041
5001,5043
5001,5049
5041,5216
5043,5052
5041,5052
5052,5259


In [21]:
%%sql

SELECT * FROM pruefen

 * sqlite:///uni.db
Done.


MatrNr,VorlNr,PersNr,Note
28106,5001,2126,1
25403,5041,2125,2
27550,4630,2137,2
25403,4052,2125,4
26830,4052,2125,2
28106,4052,2125,3
29555,4052,2125,1
26830,5041,2125,1
28106,5041,2125,4
29555,5041,2125,1


![ER Diagramm](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=hochschule-er.png)

# Aufgabe 1 ✅

Welche Studenten haben in der Prüfung der Vorlesung "Logik" mehr Punkte als in der Vorlesung "Ethik"

In [37]:
%%sql

SELECT s.MatrNr 'Matrikelnr.',
s.Name 'Student',
s.Semester 'Semester',
(SELECT p.Note
FROM pruefen p
JOIN Vorlesungen v
ON(v.VorlNr = p.VorlNr)
WHERE p.MatrNr = s.MatrNr AND v.Titel = 'Logik') 'Logik',
(SELECT p.Note
FROM pruefen p
JOIN Vorlesungen v
ON(v.VorlNr = p.VorlNr)
WHERE p.MatrNr = s.MatrNr AND v.Titel = 'Ethik') 'Ethik'
FROM Studenten s
WHERE ethik IS NOT NULL AND logik IS NOT NULL
GROUP BY s.Name
HAVING Ethik > Logik

 * sqlite:///uni.db
Done.


Matrikelnr.,Student,Semester,Logik,Ethik
28106,Carnap,3,3,4


# Aufgabe 2 ✅

Geben Sie Matrikelnummer, name und durchschnittsnote von Studenten aus, deren Durschnittsnote besser als 3 ist.

In [34]:
%%sql

SELECT s.Name 'Student', s.MatrNr 'Matrikelnr.', AVG(p.Note) 'Durschnittenote'
FROM Studenten s
JOIN pruefen p
USING(MatrNr)
GROUP BY MatrNr
HAVING Durschnittenote < 3
ORDER BY Durschnittenote



 * sqlite:///uni.db
Done.


Student,Matrikelnr.,Durschnittenote
Feuerbach,29555,1.0
Aristoxenos,26830,1.5
Schopenhauer,27550,2.0
Carnap,28106,2.6666666666666665


# Aufgabe 3 ✅

Welche Studenten haben in welchem Fach mit welcher Note bereits an mindestens einer Prüfung teilgenommen

In [50]:
%%sql

SELECT ROW_NUMBER() OVER (ORDER BY s.Name) 'Index',
s.Name 'Student', s.MatrNr 'Matrikelnr', v.Titel 'Vorlesung', p.Note
FROM Studenten s
JOIN pruefen p
USING(MatrNr)
JOIN Vorlesungen v
USING(VorlNr)
ORDER BY s.Name

 * sqlite:///uni.db
Done.


Index,Student,Matrikelnr,Vorlesung,Note
1,Aristoxenos,26830,Logik,2
2,Aristoxenos,26830,Ethik,1
3,Carnap,28106,Grundzuege,1
4,Carnap,28106,Logik,3
5,Carnap,28106,Ethik,4
6,Feuerbach,29555,Logik,1
7,Feuerbach,29555,Ethik,1
8,Jonas,25403,Ethik,2
9,Jonas,25403,Logik,4
10,Schopenhauer,27550,Die 3 Kritiken,2


# Aufgabe 4 ✅

Welche Studenten haben schon Prüfungen gesammelt?

In [52]:
%%sql

SELECT s.Name 'Student', s.MatrNr 'Matrikelnr', s.Semester
FROM Studenten s
JOIN pruefen p
USING(MatrNr)
GROUP BY s.Name
ORDER BY s.Name ASC

 * sqlite:///uni.db
Done.


Student,Matrikelnr,Semester
Aristoxenos,26830,8
Carnap,28106,3
Feuerbach,29555,2
Jonas,25403,12
Schopenhauer,27550,6


# Aufgabe 5 ✅

Geben Sie alle Informationen zu jedem Studierenden aus. Darüber hinaus geben Sie noch die Anzahl der aktuell gehörten Kurse, die Anzahl der bereits gehörten Kurse und die durschnittsnote aus. 

In [79]:
%%sql

SELECT s.*,
(SELECT COUNT(h.MatrNr) FROM hoeren h WHERE h.MatrNr = s.MatrNr) 'AnzahlVL',
(SELECT COUNT(p.MatrNr) FROM pruefen p WHERE p.MatrNr = s.MatrNr) 'AnzahlPR',
(SELECT AVG(p.Note) FROM pruefen p WHERE p.MatrNr = s.MatrNr) 'Durchschnittnote'
FROM Studenten s
ORDER BY s.Name

 * sqlite:///uni.db
Done.


MatrNr,Name,Semester,AnzahlVL,AnzahlPR,Durchschnittnote
26830,Aristoxenos,8,0,2,1.5
28106,Carnap,3,4,3,2.6666666666666665
29555,Feuerbach,2,2,2,1.0
26120,Fichte,10,1,0,None
25403,Jonas,12,1,2,3.0
27550,Schopenhauer,6,2,1,2.0
29120,Theophrastos,2,3,0,None
24002,Xenokrates,18,0,0,None


# Aufgabe 6 ✅

Wie viele Professoren heißen was mit "krates" am Ende?

In [80]:
%%sql

SELECT COUNT(*) 'Occurence'
FROM Professoren
WHERE Name LIKE '%krates'

 * sqlite:///uni.db
Done.


Occurence
1


# Aufgabe 7 ✅
Welche Studenten wurden von Sokrates geprüft?

In [83]:
%%sql

SELECT stu.*
FROM Studenten stu
JOIN pruefen pru
USING(MatrNr)
JOIN Professoren prof
USING(PersNr)
WHERE prof.Name = 'Sokrates'
GROUP BY stu.Name

 * sqlite:///uni.db
Done.


MatrNr,Name,Semester
26830,Aristoxenos,8
28106,Carnap,3
29555,Feuerbach,2
25403,Jonas,12


# Aufgabe 8 ✅

Welche Studenten hören weder eine Vorlesung, noch haben eine Prüfung abgelegt?

In [91]:
%%sql

SELECT s.*,
(SELECT COUNT(h.MatrNr) FROM hoeren h WHERE h.MatrNr = s.MatrNr) 'AnzahlVL',
(SELECT COUNT(p.MatrNr) FROM pruefen p WHERE p.MatrNr = s.MatrNr) 'AnzahlPR'
FROM Studenten s
GROUP BY s.Name
HAVING AnzahlVL = 0 AND AnzahlPR = 0

 * sqlite:///uni.db
Done.


MatrNr,Name,Semester,AnzahlVL,AnzahlPR
24002,Xenokrates,18,0,0


# Aufgabe 10 ✅

Welche Stundenten haben noch nie einen Kurs von Sokrates belegt?

In [110]:
%%sql

SELECT stu.*
FROM Studenten stu
WHERE stu.MatrNr NOT IN (
    SELECT pru.MatrNr
    FROM pruefen pru
    JOIN Professoren prof
    USING(PersNr)
    WHERE prof.Name = 'Sokrates'
) AND stu.MatrNr NOT IN (
    SELECT hoe.MatrNr
    FROM hoeren hoe
    JOIN Vorlesungen vor
    USING(VorlNr)
    JOIN Professoren prof
    ON(vor.gelesenVon = prof.PersNr)
    WHERE prof.Name = 'Sokrates'
)
ORDER BY stu.Name ASC

 * sqlite:///uni.db
Done.


MatrNr,Name,Semester
26120,Fichte,10
24002,Xenokrates,18


# Aufgabe 11 ✅

Suche Studenten (id, name, durchschnittliche Punktzahl), die zwei oder mehr Kurse nicht bestanden haben

In [115]:
%%sql

SELECT stu.MatrNr, stu.Name, AVG(pru.Note) 'Durchschnittnote'
FROM Studenten stu
JOIN pruefen pru
USING(MatrNr)
WHERE pru.Note > 4
GROUP BY stu.MatrNr
HAVING COUNT(pru.MatrNr) > 1


 * sqlite:///uni.db
Done.


MatrNr,Name,Durchschnittnote


# Aufgabe 12 ✅

Welche Studenten sind in Ethik durchgefallen? Sortieren Sie nach Matirkelnummer absteigend.

In [123]:
%%sql

SELECT stu.*, pru.Note
FROM Studenten stu
JOIN pruefen pru
USING(MatrNr)
JOIN Vorlesungen vor
USING(VorlNr)
WHERE vor.Titel = 'Ethik' AND pru.Note > 4
ORDER BY stu.MatrNr DESC

 * sqlite:///uni.db
Done.


MatrNr,Name,Semester,Note


# Aufgabe 13 ✅

Zeige für jeden Studenten die Noten aller Kurse im Vergleich zur durchschnittlich erreichten Punktzahl in diesem Kurs

In [138]:
%%sql

WITH durschnittnote(VorlNr, Note) AS (
    SELECT VorlNr, AVG(Note)
    FROM pruefen
    GROUP BY VorlNr
)

SELECT stu.*, vor.Titel, pru.Note, dsn.Note 'Durchschnittnote'
FROM Studenten stu
JOIN pruefen pru
USING(MatrNr)
JOIN Vorlesungen vor
USING(VorlNr)
JOIN durschnittnote dsn
USING(VorlNr)
ORDER BY stu.MatrNr

 * sqlite:///uni.db
Done.


MatrNr,Name,Semester,Titel,Note,Durchschnittnote
25403,Jonas,12,Logik,4,2.5
25403,Jonas,12,Ethik,2,2.0
26830,Aristoxenos,8,Logik,2,2.5
26830,Aristoxenos,8,Ethik,1,2.0
27550,Schopenhauer,6,Die 3 Kritiken,2,2.0
28106,Carnap,3,Logik,3,2.5
28106,Carnap,3,Grundzuege,1,1.0
28106,Carnap,3,Ethik,4,2.0
29555,Feuerbach,2,Logik,1,2.5
29555,Feuerbach,2,Ethik,1,2.0


# Aufgabe 14

Beste, Schlechteste und Durchschnittsnote pro Kurs

In [147]:
%%sql

WITH beste(VorlNr, Beste, Schlechte, Durchschnitt) AS (
    SELECT VorlNr, MIN(Note), MAX(Note), AVG(Note)
    FROM pruefen
    GROUP BY VorlNr
)

SELECT vor.VorlNr, vor.Titel, beste.Note
FROM Vorlesungen vor
LEFT JOIN pruefen pru
USING(VorlNr)
LEFT JOIN beste
USING(VorlNr)
GROUP BY VorlNr


 * sqlite:///uni.db
Done.


VorlNr,Titel,Note
4052,Logik,1
4630,Die 3 Kritiken,2
5001,Grundzuege,1
5022,Glaube und Wissen,None
5041,Ethik,1
5043,Erkenntnistheorie,None
5049,Maeeutik,None
5052,Wissenschaftstheorie,None
5216,Bioethik,None
5259,Der Wiener Kreis,None


In [150]:
%%sql

WITH beste(VorlNr, Beste, Schlechte, Durchschnitt) AS (
    SELECT VorlNr, MIN(Note), MAX(Note), AVG(Note)
    FROM pruefen
    GROUP BY VorlNr
)

SELECT * FROM beste

 * sqlite:///uni.db
Done.


VorlNr,Beste,Schlechte,Durchschnitt
4052,1,4,2.5
4630,2,2,2.0
5001,1,1,1.0
5041,1,4,2.0


# Aufgabe 15

Erstellen Sie ein Ranking der besten Studenten pro Kurs. Geben Sie den Namen des Kurses, den Namen des Studenten, dessen Note und dessen Ranking an.

# Aufgabe 16

Finden Sie die beiden besten Studenten pro Vorlesung